# **사전 준비**

In [ ]:
%%capture --no-stderr
%pip install -U langgraph
%pip install -U langchain-openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# 실습에 사용할 그래프의 상태 정의
# TypedDict = 타입 힌트

# Annotated는 런타임에 실제 동작하지 않고, 타입 검사기와 프레임워크에게 ‘설명/주석’ 역할을 하는 문법
# → “messages는 list 타입이다 그리고 add_messages라는 규칙이 붙어 있다”
# Python 입장에서는 단순 주석
# LangGraph 입장에서는 병합 방식 지시

class State(TypedDict):
  messages: Annotated[list, add_messages]

# 실습에서 사용할 그래프 인스턴스 생성
graph_builder = StateGraph(State)

# **챗봇 노드**

In [ ]:
from langchain_openai import ChatOpenAI

# 오픈 AI 클라이언트 정의
llm = ChatOpenAI(model="gpt-4o-mini")

# 오픈AI를 호출하여 응답을 받아온 뒤 상태값에 저장하여 반환하는 챗봇 함수 정의
def chatbot(state: State):
  return {"messages": [llm.invoke(state["messages"])]}

# 챗봇 노드 정의
graph_builder.add_node("chatbot", chatbot)

In [ ]:
from langgraph.graph import StateGraph, START, END

# 진입 지점
graph_builder.add_edge(START, "chatbot")

# 종료 지점
graph_builder.add_edge("chatbot", END)

In [ ]:
graph = graph_builder.compile()

In [ ]:
# 무한 루프
while True:
  # 사용자의 질의 입력받기
  # input() 함수는 사용자 입력을 기다림
  # "User: " 라는 안내 문구를 보여주고 입력받은 값을 user_input 변수에 저장
  user_input = input("User: ")

  # 사용자가 quit 또는 exit, q를 입력한다면 루프 종료
  if user_input.lower() in ["quit", "exit", "q"]:
    print("Goodbye!")
    break

  # 사용자의 입력을 그래프에 전달하여 정의된 흐름 실행
  # graph.stream() → LangGraph 그래프 실행
  # ("user", user_input) → 튜플로 “사용자가 입력했다” 표시
  # 결괏값 event에 저장
  # {"messages": ("user", user_input)} 정의한 State
  for event in graph.stream({"messages": ("user", user_input)}):
    for value in event.values():
      print("Assistant:", value["messages"][-1].content)

# **그래프 시각화**

In [ ]:
from IPython.display import Image, dsiplay

display(Image(graph.get_graph().draw_mermaid_png()))

# **Tavily 검색 엔진 세팅**

In [ ]:
%%capture --no-stderr
%pip install -U tavily-python
%pip install -U langchain_community

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Tavily 검색 엔진을 도구로 정의
tool = TavilySearchResults(max_results=2)
tools = [tool]

# 호출 예시
tool.invoke("내일 대한민국 서울의 날씨는?")

# **외부 검색 도구 노드**

In [ ]:
from typing import Annotated
from langchain_openai import ChatOpneAI
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages

# 그래프 상태 정의
class State(TypedDict):
  messages: Annotated[list, add_messages]

# 그래프 정의
graph_builder = StateGraph(State)

# 오픈AI 클라이언트 정의
llm = ChatOpenAI(model="gpt-4o-mini")
# 오픈AI 클라이언트에 Tavily 검색 엔진 도구 할당
# llm이 판별할 수 있도록 툴 정보를 바인딩
# 툴 내부에 툴의 이름, 기능 등이 정의?
llm_with_tools = llm.bind_tools(tools)

# 챗봇 함수 정의
def chatbot(state: State):
  return {"messages": [llm_with_tools.invoke(state["messages"])]}

# 그래프에 챗봇 노드 추가
graph_builder.add_node("chatbot", chatbot)

In [ ]:
import json

# 도구 실행 결과를 담는 메시지
from langchain_core.messages import ToolMessage

# 도구 노드로 사용할 클래스
class BasicToolNode:
  # 도구 노드에서 사용할 초기 파라미터 정의
  # -> None: = 이 함수는 값을 반환하지 않는다는 타입 힌트
  def __init__(self, tools: list) -> None:
    # tools를 받아 도구 이름이 키고 값이 툴인 딕셔너리 생성
    self.tools_by_name = {tool.name: tool for tool in tools}

  # 도구 노드가 호출될 때의 행동 정의
  # __call__ = 객체를 함수처럼 호출 가능 -> tool_node(inputs) -> 내부적으로 tool_node.__call__(inputs)
  def __call__(self, inputs: dict):
    # 입력된 상태의 가장 마지막 메시지 획득
    # 딕셔너리에서 값을 꺼낼때 null 에러를 방지하기 위해
    # "messages"가 있으면 해당 값 리턴, 없으면 [] 리턴
    if messages := inputs.get("messages", []):
      message = messages[-1]
    else:
      raise ValueError("No message found in input")

    # 메시지의 tool_calls에 도구 정보가 존재한다면 이를 활용해 도구 호출
    outputs = []

    for tool_call in message.tool_calls:
      tool_result = self.tools_by_name[tool_call["name"]].invoke(tool_call["args"])

      # 도구 호출의 결과물을 ToolMessage로 정의하여 출력값에 저장
      # 도구 결과를 다시 LLM에게 알려줘야 하기 때문에 형식을 맞춤
      outputs.append(
          ToolMessage(
              # tool_result의 값을 json 문자열로 변환
              # ensure_ascii=False = 한글 깨짐 방지 옵션
              content=json.dumps(tool_result, ensure_ascii=False),
              # 어떤 도구의 결과인지를 알려주기 위함
              name=tool_call["name"],
              # 어떤 요청에 대한 응답인지 알려주기 위함
              # 한 번에 여러 도구를 호출할 수 있기 때문에 ID로 짝 맞춤
              tool_call_id=tool_call["id"],
          )
      )

    # 출력값을 상태값 형식에 맞춰 변환
    return {"messages": outputs}

# 도구 노드 정의
tool_node = BasicToolNode(tools=[tool])

# 도구 노드 그래프에 추가
graph_builder.add_node("tools", tool_node)

In [ ]:
from typing import Literal

# 도구 노드 호출 여부를 결정하는 함수 정의
# Literal["tools", "__end__"] -> 반환값 타입 힌트 -> tools나 __end__만 반환
def route_tools(state: State,) ->  Literal["tools", "__end__"]:
  # 상태값의 가장 최근 메시지를 정의
  # state가 list라면
  if isinstance(state, list):
    ai_message = state[-1]
  # state가 딕셔너리라면 messages 추출하고 없으면 빈 리스트
  elif messages := state.get("messages", []):
    ai_message = messages[-1]
  else:
    raise ValueError(f"No messages found in input state to tool_edge {state}")

  # 가장 최근 메시지에 tool_calls 속성이 있다면 tools 노드를, 아니라면 종료 지정을 반환
  # hasattr -> 객체에 그 이름의 속성이 있으면 true 반환
  if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
    return "tools"

  return "__end__"

# 챗봇 노드에 조건부 에지를 정의
# chatbot 노드 실행 후
# route_tools 함수 실행
# 반환값에 따라 다음 노드 결정
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    {"tools": "tools", "__end__": "__end__"},
)

In [ ]:
# 도구 노드와 챗봇 노드 연결
graph_builder.add_edge("tools", "chatbot")

# 진입 지점으로 챗봇 노드 지정
graph_builder.add_edge(START, "chatbot")

In [ ]:
# 그래프 컴파일
graph = graph_builder.compile()
# 그래프 이미지화
display(Image(graph.get_graph().draw_mermaid_png()))

# **스트리밍**

In [ ]:
from langchain_core.messages import BaseMessage

while True:
  # 사용자 질문 입력받기
  user_inpur = input("User: ")
  print("User:", user_input)

  if user_input.lower() in ["quit", "exit", "q"]:
    print("Goodbye!")
    break

  # 업데이트된 내용을 확인할 수 있는 그래프 스트리밍 정의
  events = graph.stream(input={"messages": [("user", user_input)]},
                        stream_mode="updates")

  # 그래프 이벤트 내의 메시지를 출력
  for event in events:
    for value in event.values():
      if isinstance(value["messages"][-1], BaseMessage):
        print("Assistant:", value["messages"][-1].content)

# **상태 저장하기**

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
from typing import Annotated

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from typing_extensions import TypedDict

from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

class State(TypedDict):
  messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

tool = TavilySearchResults(max_results=2)
tools = [tool]

llm = ChatOpenAI(model="gpt-40-mini")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
  return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)

# 미리 빌드된 도구 노드
tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

# 미리 빌드된 조건부 에지
# "chatbot" -> 출발 노드
# tools_condition -> 상태를 보고 어디로 갈지 결정하는 함수
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)

# tools -> 시작
# chatbot -> 도착
# tools 실행 후 chatbot으로 돌아오라는 의미
graph_builder.add_edge("tools", "chatbot")

# START -> 그래프 진입 지점
graph_builder.add_edge(START, "chatbot")

# 체크포인터를 지정하여 그래프를 컴파일
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
from iPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
user_input = "안녕! 내 이름은 오해원이야."

events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)

for event in events:
  event["messages"][-1].pretty_print()

In [ ]:
user_input = "내 이름을 기억하니?"

events = graph.stream(
    {"messages": [("user", user_input)]}
)

In [ ]:
events = graph.stream(
    {"messages": [("user", user_input)]},
    # thread_id = 2로 변경
    {"configurable": {"thread_id": "2"}},
)

for event in events:
  event["messages"][-1].pretty_print()

In [ ]:
snapshot = graph.get_state(config)

print(snapshot)

# **루프 개입하기**

In [ ]:
graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_before=["tools"],
)

In [ ]:
user_input = "지금 서울 날씨 어때?"
config = {"configurable": {"thread_id": "2"}}
events = graph.stream(
    {"messages": [("user", user_input)]},
    config,
    stream_mode="values"
)

for event in events:
  if "messages" in event:
    event["messages"][-1].pretty_print()

In [ ]:
snapshot = graph.get_state(config)

print(snapshot.next)

# **state 업데이트**

In [ ]:
from langchain_core.messages import AIMessage

# 최근 메시지
existing_message = snapshot.values["messages"][-1]
# 최근 메시지의 id
existing_message_id = existing_message.tool_calls[0]["id"]

# 강제할 응답 정의
answer = "서울의 날씨는 매우 맑아요."

# 강제할 응답을 포함한 메시지 상태 정의
new_messages = [
    ToolMessage(content=answer, tool_call_id=existing_message_id),
    AIMessage(content=answer)
]

# 그래프 상태를 새로 작성한 메시지 상태로 변경
graph.update_state(
    config,
    {"messages": new_messages},
)

In [ ]:
print("\n\nLast 2 messages;")
print(graph.get_state(config).values["messages"][-2:])